In [1]:
import os, time
start = time.time()
very_start = time.time()

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [3]:
import dask
from dask.distributed import Client, wait, LocalCluster
import dask.dataframe as dd

In [4]:

client = Client(n_workers=1, 
                       threads_per_worker=96,
                       memory_limit='480GB',ip='10.1.0.xxx')
dask.config.set(shuffle='disk')
dask.config.set({'temporary_directory': '/path/to/dask_tmp'})

# Load Train

In [5]:
%%time
path = '/path/to/bin/tmp/'
train = dd.read_parquet(f'/path/to/dask_tmp/train_10_raw/*.parquet')#,dtypes=dtypes)
valid = dd.read_parquet(f'/path/to/dask_tmp/valid_10_raw/*.parquet')#,dtypes=dtypes)

CPU times: user 106 ms, sys: 14.5 ms, total: 121 ms
Wall time: 112 ms


In [6]:
cols_drop = ['tweet','tweet_nortsign']
train = train.drop(cols_drop,axis=1)
valid = valid.drop(cols_drop,axis=1)

In [7]:
dtype_names = ['count_words', 'count_char', 'count_ats']
for col in dtype_names:
    train[col] = train[col].astype('int32')
    valid[col] = valid[col].astype('int32')

In [8]:
label_names = ['reply_timestamp', 'retweet_timestamp', 'retweet_with_comment_timestamp', 'like_timestamp']
DONT_USE = ['tweet_id','tweet_timestamp','engaged_with_user_account_creation','enaging_user_account_creation','engage_time',
            'fold','enaging_user_id','engaged_with_user_id', 'dt_dow',
            'engaged_with_user_account_creation', 'enaging_user_account_creation', 'elapsed_time',
             'present_links','present_domains']
DONT_USE += label_names
features = [c for c in train.columns if c not in DONT_USE]

# Train Model Validate
We will train on random 10% of first 5 days and validation on last 2 days

In [9]:
%%time

SAMPLE_RATIO = 0.5
SEED = 1

if SAMPLE_RATIO < 1.0:
    print(len(train))
    train = train.sample(frac=SAMPLE_RATIO,random_state=42)
    train, = dask.persist(train)
    train.head()
    print(len(train))

train = train.compute()
# train = train.compute()
Y_train = train[label_names]
train = train.drop(label_names,axis=1)

features = [c for c in train.columns if c not in DONT_USE]
print('Using %i features:'%(len(features)),train.shape[1])
np.asarray(features)

21749425
10874713
Using 121 features: 121
CPU times: user 11.2 s, sys: 12.6 s, total: 23.8 s
Wall time: 2min 41s


array(['hashtags', 'present_media', 'tweet_type', 'language',
       'engaged_with_user_follower_count',
       'engaged_with_user_following_count',
       'engaged_with_user_is_verified', 'enaging_user_follower_count',
       'enaging_user_following_count', 'enaging_user_is_verified',
       'engagee_follows_engager', 'len_hashtags', 'len_domains',
       'len_links', 'dt_hour', 'dt_minute', 'dt_second', 'count_words',
       'count_char', 'tw_uhash', 'tw_hash', 'count_ats', 'hash0', 'hash1',
       'tw_freq_hash', 'tw_first_word', 'tw_second_word', 'tw_last_word',
       'tw_llast_word', 'tw_len', 'TE_present_media_reply_timestamp',
       'TE_tweet_type_reply_timestamp', 'TE_language_reply_timestamp',
       'TE_engaged_with_user_id_reply_timestamp',
       'TE_enaging_user_id_reply_timestamp',
       'TE_present_media_retweet_timestamp',
       'TE_tweet_type_retweet_timestamp', 'TE_language_retweet_timestamp',
       'TE_engaged_with_user_id_retweet_timestamp',
       'TE_enaging_

In [10]:
SAMPLE_RATIO = 0.052 # VAL SET NOW SIZE OF TEST SET
SEED = 1
if SAMPLE_RATIO < 1.0:
    print(len(valid))
    valid = valid.sample(frac=SAMPLE_RATIO,random_state=42)
    valid, = dask.persist(valid)
    valid.head()
    print(len(valid))
    
valid = valid.compute()
Y_valid = valid[label_names]
valid = valid.drop(label_names,axis=1)

8585006
446420


In [31]:
import sys
sys.path.append("..")
from Models.GBM.LightGBM import LightGBM
params = {
    'num_leaves': 61.35765882069168,
    'learning_rate': 0.07266779287696,
    'max_depth': 28,
    'lambda_l1': 50.0,
    'lambda_l2': 50.0,
    'colsample_bynode': 0.8,
    'colsample_bytree': 0.4,
    'bagging_fraction': 0.8,
    'bagging_freq': 7,
    'max_bin': 163.51837199855655,
    'min_data_in_leaf': 1282.3912530172006
}

LGBM0 = LightGBM(
    objective='binary',
    num_threads=94,
    num_iterations=1500,
    early_stopping_rounds=20,
    **params,
)
LGBM1 = LightGBM(
    objective='binary',
    num_threads=94,
    num_iterations=1500,
    early_stopping_rounds=20,
    **params,
)
LGBM2 = LightGBM(
    objective='binary',
    num_threads=94,
    num_iterations=1500,
    early_stopping_rounds=20,
    **params,
)
LGBM3 = LightGBM(
    objective='binary',
    num_threads=94,
    num_iterations=1500,
    early_stopping_rounds=20,
    **params,
)
model2 = [LGBM0,LGBM1,LGBM2,LGBM3]

In [32]:
if train.columns.duplicated().sum()>0:
    raise Exception(f'duplicated!: { train.columns[train.columns.duplicated()] }')
print('no dup :) ')
print(f'X_train.shape {train.shape}')
print(f'X_valid.shape {valid.shape}')

no dup :) 
X_train.shape (10874713, 121)
X_valid.shape (446420, 121)


In [33]:
%%time

for col in train.columns:
    if train[col].dtype=='bool':
        train[col] = train[col].astype('int8')
        valid[col] = valid[col].astype('int8')

CPU times: user 1.25 ms, sys: 2 µs, total: 1.25 ms
Wall time: 1.27 ms


In [34]:
train.head()

,hashtags,present_media,tweet_type,language,engaged_with_user_follower_count,engaged_with_user_following_count,engaged_with_user_is_verified,enaging_user_follower_count,enaging_user_following_count,enaging_user_is_verified,engagee_follows_engager,len_hashtags,len_domains,len_links,dt_hour,dt_minute,dt_second,count_words,count_char,tw_uhash,tw_hash,count_ats,hash0,hash1,tw_freq_hash,tw_first_word,tw_second_word,tw_last_word,tw_llast_word,tw_len,TE_present_media_reply_timestamp,TE_tweet_type_reply_timestamp,TE_language_reply_timestamp,TE_engaged_with_user_id_reply_timestamp,TE_enaging_user_id_reply_timestamp,TE_present_media_retweet_timestamp,TE_tweet_type_retweet_timestamp,TE_language_retweet_timestamp,TE_engaged_with_user_id_retweet_timestamp,TE_enaging_user_id_retweet_timestamp,TE_present_media_retweet_with_comment_timestamp,TE_tweet_type_retweet_with_comment_timestamp,TE_language_retweet_with_comment_timestamp,TE_engaged_with_user_id_retweet_with_comment_timestamp,TE_enaging_user_id_retweet_with_comment_timestamp,TE_present_media_like_timestamp,TE_tweet_type_like_timestamp,TE_language_like_timestamp,TE_engaged_with_user_id_like_timestamp,TE_enaging_user_id_like_timestamp,TE_present_domains_language_engagee_follows_engager_tweet_type_present_media_engaged_with_user_is_verified_reply_timestamp,TE_engaged_with_user_id_tweet_type_language_reply_timestamp,TE_tw_first_word_tweet_type_language_reply_timestamp,TE_tw_last_word_tweet_type_language_reply_timestamp,TE_hash0_tweet_type_language_reply_timestamp,TE_hash1_tweet_type_language_reply_timestamp,TE_tw_uhash_tweet_type_language_reply_timestamp,TE_tw_hash_reply_timestamp,TE_tw_freq_hash_reply_timestamp,TE_present_media_tweet_type_language_engaged_with_user_is_verified_enaging_user_is_verified_engagee_follows_engager_reply_timestamp,TE_present_domains_present_media_tweet_type_language_reply_timestamp,TE_present_links_present_media_tweet_type_language_reply_timestamp,TE_hashtags_present_media_tweet_type_language_reply_timestamp,TE_present_domains_language_engagee_follows_engager_tweet_type_present_media_engaged_with_user_is_verified_retweet_timestamp,TE_engaged_with_user_id_tweet_type_language_retweet_timestamp,TE_tw_first_word_tweet_type_language_retweet_timestamp,TE_tw_last_word_tweet_type_language_retweet_timestamp,TE_hash0_tweet_type_language_retweet_timestamp,TE_hash1_tweet_type_language_retweet_timestamp,TE_tw_uhash_tweet_type_language_retweet_timestamp,TE_tw_hash_retweet_timestamp,TE_tw_freq_hash_retweet_timestamp,TE_present_media_tweet_type_language_engaged_with_user_is_verified_enaging_user_is_verified_engagee_follows_engager_retweet_timestamp,TE_present_domains_present_media_tweet_type_language_retweet_timestamp,TE_present_links_present_media_tweet_type_language_retweet_timestamp,TE_hashtags_present_media_tweet_type_language_retweet_timestamp,TE_present_domains_language_engagee_follows_engager_tweet_type_present_media_engaged_with_user_is_verified_retweet_with_comment_timestamp,TE_engaged_with_user_id_tweet_type_language_retweet_with_comment_timestamp,TE_tw_first_word_tweet_type_language_retweet_with_comment_timestamp,TE_tw_last_word_tweet_type_language_retweet_with_comment_timestamp,TE_hash0_tweet_type_language_retweet_with_comment_timestamp,TE_hash1_tweet_type_language_retweet_with_comment_timestamp,TE_tw_uhash_tweet_type_language_retweet_with_comment_timestamp,TE_tw_hash_retweet_with_comment_timestamp,TE_tw_freq_hash_retweet_with_comment_timestamp,TE_present_media_tweet_type_language_engaged_with_user_is_verified_enaging_user_is_verified_engagee_follows_engager_retweet_with_comment_timestamp,TE_present_domains_present_media_tweet_type_language_retweet_with_comment_timestamp,TE_present_links_present_media_tweet_type_language_retweet_with_comment_timestamp,TE_hashtags_present_media_tweet_type_language_retweet_with_comment_timestamp,TE_present_domains_language_engagee_follows_engager_tweet_type_present_media_engaged_with_user_is_verified_like_timestamp,TE_engaged_with_user

In [35]:
#label_names = ['reply_timestamp', 'retweet_timestamp', 'retweet_with_comment_timestamp', 'like_timestamp']

In [36]:
%%time
# TRAIN AND VALIDATE

NROUND = 500
VERBOSE_EVAL = 50
#ESR = 50
LR = [0.1,0.03,0.07,0.01]

#Like
#xgb_parms['learning_rate'] = LR[TARGET_id]    
oof = np.zeros((len(valid),len(label_names)))
preds = []

for i in range(4):
    name = label_names[i]
    print('#'*25);print('###',name);print('#'*25)

    start = time.time(); print('Training...')
#     model[i].fit(X=train, Y=Y_train.iloc[:, i],
#          categorical_feature=set([]))
    model2[i].fit(X=train, Y=Y_train.iloc[:, i], X_val=valid, Y_val=Y_valid.iloc[:, i],
         categorical_feature=set([]))
    print('Training took %.1f seconds'%(time.time()-start))
        
    start = time.time(); print('Predicting...')
    #Y_valid[f'pred_{name}'] = xgb.dask.predict(client,model,valid)
    oof[:, i] += model2[i].get_prediction(valid)
    #preds.append(xgb.dask.predict(client,model,valid))
    print('Took %.1f seconds'%(time.time()-start))

#########################
### reply_timestamp
#########################
Training...


/home/xianyang/sw/miniconda3/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/xianyang/sw/miniconda3/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's binary_logloss: 0.130751
Training until validation scores don't improve for 20 rounds
[2]	valid_0's binary_logloss: 0.127487
[3]	valid_0's binary_logloss: 0.125697
[4]	valid_0's binary_logloss: 0.123971
[5]	valid_0's binary_logloss: 0.12255
[6]	valid_0's binary_logloss: 0.121293
[7]	valid_0's binary_logloss: 0.119757
[8]	valid_0's binary_logloss: 0.118834
[9]	valid_0's binary_logloss: 0.118142
[10]	valid_0's binary_logloss: 0.117543
[11]	valid_0's binary_logloss: 0.116897
[12]	valid_0's binary_logloss: 0.116358
[13]	valid_0's binary_logloss: 0.11588
[14]	valid_0's binary_logloss: 0.115477
[15]	valid_0's binary_logloss: 0.114737
[16]	valid_0's binary_logloss: 0.114123
[17]	valid_0's binary_logloss: 0.113837
[18]	valid_0's binary_logloss: 0.113549
[19]	valid_0's binary_logloss: 0.113044
[20]	valid_0's binary_logloss: 0.112627
[21]	valid_0's binary_logloss: 0.112236
[22]	valid_0's binary_logloss: 0.112046
[23]	valid_0's binary_logloss: 0.111872
[24]	valid_0's binary_logloss:

[203]	valid_0's binary_logloss: 0.107188
[204]	valid_0's binary_logloss: 0.107183
[205]	valid_0's binary_logloss: 0.10718
[206]	valid_0's binary_logloss: 0.107177
[207]	valid_0's binary_logloss: 0.107175
[208]	valid_0's binary_logloss: 0.107172
[209]	valid_0's binary_logloss: 0.107167
[210]	valid_0's binary_logloss: 0.107164
[211]	valid_0's binary_logloss: 0.107162
[212]	valid_0's binary_logloss: 0.107159
[213]	valid_0's binary_logloss: 0.107157
[214]	valid_0's binary_logloss: 0.107155
[215]	valid_0's binary_logloss: 0.107151
[216]	valid_0's binary_logloss: 0.107148
[217]	valid_0's binary_logloss: 0.107146
[218]	valid_0's binary_logloss: 0.107143
[219]	valid_0's binary_logloss: 0.107141
[220]	valid_0's binary_logloss: 0.107139
[221]	valid_0's binary_logloss: 0.107137
[222]	valid_0's binary_logloss: 0.107134
[223]	valid_0's binary_logloss: 0.107131
[224]	valid_0's binary_logloss: 0.107127
[225]	valid_0's binary_logloss: 0.107126
[226]	valid_0's binary_logloss: 0.107122
[227]	valid_0's b

[404]	valid_0's binary_logloss: 0.10679
[405]	valid_0's binary_logloss: 0.10679
[406]	valid_0's binary_logloss: 0.106789
[407]	valid_0's binary_logloss: 0.106787
[408]	valid_0's binary_logloss: 0.106786
[409]	valid_0's binary_logloss: 0.106785
[410]	valid_0's binary_logloss: 0.106784
[411]	valid_0's binary_logloss: 0.106785
[412]	valid_0's binary_logloss: 0.106783
[413]	valid_0's binary_logloss: 0.106781
[414]	valid_0's binary_logloss: 0.10678
[415]	valid_0's binary_logloss: 0.106779
[416]	valid_0's binary_logloss: 0.106778
[417]	valid_0's binary_logloss: 0.106778
[418]	valid_0's binary_logloss: 0.106777
[419]	valid_0's binary_logloss: 0.106777
[420]	valid_0's binary_logloss: 0.106776
[421]	valid_0's binary_logloss: 0.106773
[422]	valid_0's binary_logloss: 0.106773
[423]	valid_0's binary_logloss: 0.106773
[424]	valid_0's binary_logloss: 0.106771
[425]	valid_0's binary_logloss: 0.10677
[426]	valid_0's binary_logloss: 0.10677
[427]	valid_0's binary_logloss: 0.10677
[428]	valid_0's binary

...

[235]	valid_0's binary_logloss: 0.0370119
[236]	valid_0's binary_logloss: 0.0370107
[237]	valid_0's binary_logloss: 0.0370107
[238]	valid_0's binary_logloss: 0.0370102
[239]	valid_0's binary_logloss: 0.0370109
[240]	valid_0's binary_logloss: 0.0370096
[241]	valid_0's binary_logloss: 0.0370099
[242]	valid_0's binary_logloss: 0.03701
[243]	valid_0's binary_logloss: 0.037009
[244]	valid_0's binary_logloss: 0.0370079
[245]	valid_0's binary_logloss: 0.0370068
[246]	valid_0's binary_logloss: 0.0370073
[247]	valid_0's binary_logloss: 0.0370069
[248]	valid_0's binary_logloss: 0.0370064
[249]	valid_0's binary_logloss: 0.0370066
[250]	valid_0's binary_logloss: 0.037006
[251]	valid_0's binary_logloss: 0.037005
[252]	valid_0's binary_logloss: 0.0370046
[253]	valid_0's binary_logloss: 0.0370048
[254]	valid_0's binary_logloss: 0.0370042
[255]	valid_0's binary_logloss: 0.0370037
[256]	valid_0's binary_logloss: 0.0370037
[257]	valid_0's binary_logloss: 0.037003
[258]	valid_0's binary_logloss: 0.037002

[431]	valid_0's binary_logloss: 0.0369253
[432]	valid_0's binary_logloss: 0.0369249
[433]	valid_0's binary_logloss: 0.0369247
[434]	valid_0's binary_logloss: 0.0369255
[435]	valid_0's binary_logloss: 0.0369256
[436]	valid_0's binary_logloss: 0.0369256
[437]	valid_0's binary_logloss: 0.0369254
[438]	valid_0's binary_logloss: 0.036924
[439]	valid_0's binary_logloss: 0.0369241
[440]	valid_0's binary_logloss: 0.0369242
[441]	valid_0's binary_logloss: 0.0369234
[442]	valid_0's binary_logloss: 0.0369231
[443]	valid_0's binary_logloss: 0.0369223
[444]	valid_0's binary_logloss: 0.0369227
[445]	valid_0's binary_logloss: 0.0369229
[446]	valid_0's binary_logloss: 0.0369225
[447]	valid_0's binary_logloss: 0.0369212
[448]	valid_0's binary_logloss: 0.0369206
[449]	valid_0's binary_logloss: 0.0369206
[450]	valid_0's binary_logloss: 0.0369204
[451]	valid_0's binary_logloss: 0.0369201
[452]	valid_0's binary_logloss: 0.0369208
[453]	valid_0's binary_logloss: 0.0369204
[454]	valid_0's binary_logloss: 0.0

[150]	valid_0's binary_logloss: 0.538202
[151]	valid_0's binary_logloss: 0.538147
[152]	valid_0's binary_logloss: 0.538102
[153]	valid_0's binary_logloss: 0.538031
[154]	valid_0's binary_logloss: 0.537989
[155]	valid_0's binary_logloss: 0.537954
[156]	valid_0's binary_logloss: 0.537903
[157]	valid_0's binary_logloss: 0.537855
[158]	valid_0's binary_logloss: 0.537841
[159]	valid_0's binary_logloss: 0.537809
[160]	valid_0's binary_logloss: 0.537771
[161]	valid_0's binary_logloss: 0.537728
[162]	valid_0's binary_logloss: 0.537697
[163]	valid_0's binary_logloss: 0.537714
[164]	valid_0's binary_logloss: 0.537672
[165]	valid_0's binary_logloss: 0.537627
[166]	valid_0's binary_logloss: 0.537595
[167]	valid_0's binary_logloss: 0.537539
[168]	valid_0's binary_logloss: 0.537503
[169]	valid_0's binary_logloss: 0.537463
[170]	valid_0's binary_logloss: 0.537409
[171]	valid_0's binary_logloss: 0.53738
[172]	valid_0's binary_logloss: 0.537343
[173]	valid_0's binary_logloss: 0.537299
[174]	valid_0's b

[351]	valid_0's binary_logloss: 0.534411
[352]	valid_0's binary_logloss: 0.534398
[353]	valid_0's binary_logloss: 0.534413
[354]	valid_0's binary_logloss: 0.534399
[355]	valid_0's binary_logloss: 0.534387
[356]	valid_0's binary_logloss: 0.534377
[357]	valid_0's binary_logloss: 0.53437
[358]	valid_0's binary_logloss: 0.534355
[359]	valid_0's binary_logloss: 0.534348
[360]	valid_0's binary_logloss: 0.534343
[361]	valid_0's binary_logloss: 0.534331
[362]	valid_0's binary_logloss: 0.534285
[363]	valid_0's binary_logloss: 0.534276
[364]	valid_0's binary_logloss: 0.53427
[365]	valid_0's binary_logloss: 0.534258
[366]	valid_0's binary_logloss: 0.534251
[367]	valid_0's binary_logloss: 0.534267
[368]	valid_0's binary_logloss: 0.534279
[369]	valid_0's binary_logloss: 0.53428
[370]	valid_0's binary_logloss: 0.534272
[371]	valid_0's binary_logloss: 0.534266
[372]	valid_0's binary_logloss: 0.534251
[373]	valid_0's binary_logloss: 0.534241
[374]	valid_0's binary_logloss: 0.534233
[375]	valid_0's bin

[552]	valid_0's binary_logloss: 0.533046
[553]	valid_0's binary_logloss: 0.533043
[554]	valid_0's binary_logloss: 0.533037
[555]	valid_0's binary_logloss: 0.533031
[556]	valid_0's binary_logloss: 0.533033
[557]	valid_0's binary_logloss: 0.533027
[558]	valid_0's binary_logloss: 0.53302
[559]	valid_0's binary_logloss: 0.533007
[560]	valid_0's binary_logloss: 0.533005
[561]	valid_0's binary_logloss: 0.533002
[562]	valid_0's binary_logloss: 0.532996
[563]	valid_0's binary_logloss: 0.532989
[564]	valid_0's binary_logloss: 0.532987
[565]	valid_0's binary_logloss: 0.532996
[566]	valid_0's binary_logloss: 0.532991
[567]	valid_0's binary_logloss: 0.532988
[568]	valid_0's binary_logloss: 0.532984
[569]	valid_0's binary_logloss: 0.532979
[570]	valid_0's binary_logloss: 0.532976
[571]	valid_0's binary_logloss: 0.532977
[572]	valid_0's binary_logloss: 0.532971
[573]	valid_0's binary_logloss: 0.532963
[574]	valid_0's binary_logloss: 0.532957
[575]	valid_0's binary_logloss: 0.532949
[576]	valid_0's b

In [37]:
yvalid = Y_valid[label_names].values

# Compute Validation Metrics

In [38]:
from sklearn.metrics import precision_recall_curve, auc, log_loss
from sklearn.metrics import average_precision_score

def compute_prauc(pred, gt):
  prec, recall, thresh = precision_recall_curve(gt, pred)
  prauc = auc(recall, prec)
  return prauc

def compute_AP(pred, gt):
  return average_precision_score(gt, pred)

def calculate_ctr(gt):
  positive = len([x for x in gt if x == 1])
  ctr = positive/float(len(gt))
  return ctr

def compute_rce(pred, gt):
    cross_entropy = log_loss(gt, pred)
    data_ctr = calculate_ctr(gt)
    strawman_cross_entropy = log_loss(gt, [data_ctr for _ in range(len(gt))])
    return (1.0 - cross_entropy/strawman_cross_entropy)*100.0

# FAST METRIC FROM GIBA
def compute_rce_fast(pred, gt):
    cross_entropy = log_loss(gt, pred)
    yt = np.mean(gt)     
    strawman_cross_entropy = -(yt*np.log(yt) + (1 - yt)*np.log(1 - yt))
    return (1.0 - cross_entropy/strawman_cross_entropy)*100.0

In [39]:
%%time
txt = ''
for i in range(4):
    ap = compute_AP(oof[:,i], yvalid[:, i])
    rce   = compute_rce_fast(oof[:,i], yvalid[:, i])
    txt_ = f"{label_names[i]:20} AP:{ap:.5f} RCE:{rce:.5f}"
    print(txt_)
    txt += txt_ + '\n'

reply_timestamp      AP:0.21591 RCE:20.68662
retweet_timestamp    AP:0.42902 RCE:23.62521
retweet_with_comment_timestamp AP:0.04663 RCE:10.69715
like_timestamp       AP:0.72792 RCE:21.26861
CPU times: user 825 ms, sys: 6.97 ms, total: 832 ms
Wall time: 808 ms


In [40]:
print('This notebook took %.1f minutes'%((time.time()-very_start)/60.))

This notebook took 42.6 minutes


In [41]:
print(f'X_valid.shape {oof.shape}')

X_valid.shape (446420, 4)
